In [21]:
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
data = pd.read_csv('C:/Users/User/Downloads/walmart-products.csv')
data.head()

,timestamp,url,final_price,sku,currency,gtin,specifications,image_urls,top_reviews,rating_stars,...,sizes,colors,seller,other_attributes,customer_reviews,ingredients,initial_price,discount,ingredients_full,categories
0,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Laura-Mercier-Cavia...,22.90,173530386,USD,7.361501e+11,"[{""name"":""Brand"",""value"":""Laura Mercier""},{""na...","[""https://i5.walmartimages.com/seo/Laura-Merci...","{""negative"":{},""positive"":{}}","{""five_stars"":2,""four_stars"":3,""two_stars"":1}",...,[],"[""Sugar Frost"",""Tuxedo""]",Wal███t.c███,"[{""name"":""Instructions"",""value"":""Apply directl...","[{""name"":""Jac███"",""rating"":5,""review"":""My only...","Cyclopentasiloxane, trimethylsiloxysilicate, s...",NaN,NaN,"[{""type"":""Ingredients"",""values"":""Cyclopentasil...","[""Beauty"",""Makeup"",""Eye Makeup"",""Eye Shadow"",""..."
1,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Exultantex-Grey-Bla...,47.88,430528189,USD,7.710779e+11,"[{""name"":""Brand"",""value"":""Exultantex""},{""name""...","[""https://i5.walmartimages.com/seo/Exultantex-...","{""negative"":{""rating"":1,""review"":""Color not ac...","{""five_stars"":47,""four_stars"":4,""one_star"":4,""...",...,"[""50\"" x 54\"""",""50\"" x 63\"""",""50\"" x 84\"""",""50...","[""Black"",""Blue"",""Green"",""Gray"",""Natural(Ivory)...",Exu███nte███ome███,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Dana"",""rating"":5,""review"":""I love th...",NaN,70.80,$22.92,NaN,"[""Home"",""Decor"",""Curtains & Window Treatments""..."
2,2024-08-24 00:00:00.000,https://www.walmart.com/ip/Jessica-London-Wome...,33.24,6013308220,USD,4.651920e+11,"[{""name"":""Features"",""value"":""Easy Care""},{""nam...","[""https://i5.walmartimages.com/seo/Jessica-Lon...","{""negative"":{},""positive"":{}}","{""five_stars"":2}",...,"[""S"",""M"",""L"",""1X"",""2X"",""3X""]","[""Aqua Sea"",""Dark Olive Green"",""Dark Sapphire""...",Ful███aut███ran███,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Bev███y F███"",""rating"":5,""review"":""V...",NaN,37.99,$4.75,NaN,"[""Clothing"",""Womens Plus"",""Plus Size Tops"",""Pl..."
3,2024-08-24 00:00:00.000,https://www.walmart.com/ip/100-Cotton-King-Per...,49.99,161657830,USD,8.407081e+11,"[{""name"":""Brand"",""value"":""Simply Put""},{""name""...","[""https://i5.walmartimages.com/asr/d88fe658-b9...","{""negative"":{""rating"":1,""review"":""[This review...","{""five_stars"":78,""four_stars"":17,""one_star"":2,...",...,"[""Queen"",""King""]","[""Beige"",""Blue"",""Gray"",""Spa Blue""]",IC ███bal███c,"[{""name"":""Fabric Care Instructions"",""value"":""M...","[{""name"":""Jes███oge███"",""rating"":5,""review"":""[...",NaN,71.99,$22.00,NaN,"[""Home"",""Bedding"",""Duvet Covers"",""King Duvet C..."
4,2024-08-25 00:00:00.000,https://www.walmart.com/ip/Disney-Boys-Graphic...,12.99,5397071399,USD,4.600086e+11,"[{""name"":""Country of Origin - Textiles"",""value...","[""https://i5.walmartimages.com/seo/Disney-Boys...","{""negative"":{""rating"":1,""review"":""This size sh...","{""five_stars"":12,""one_star"":1,""three_stars"":1,...",...,"[""2T"",""3T"",""3T-4T"",""3-4 Years"",""4-5 Years"",""4-...","[""1#Yellow Micky"",""2#Pink Minnie"",""3#Blue Dona...",Pat███,NaN,"[{""name"":""Mary"",""rating"":5,""review"":""Love thes...",NaN,29.00,$16.01,NaN,"[""Clothing"",""Kids Clothing"",""Boys Clothing"",""B..."


### Basic Info

In [23]:
data.columns

Index(['timestamp', 'url', 'final_price', 'sku', 'currency', 'gtin',
       'specifications', 'image_urls', 'top_reviews', 'rating_stars',
       'related_pages', 'available_for_delivery', 'available_for_pickup',
       'brand', 'breadcrumbs', 'category_ids', 'review_count', 'description',
       'product_id', 'product_name', 'review_tags', 'category_url',
       'category_name', 'category_path', 'root_category_url',
       'root_category_name', 'upc', 'tags', 'main_image', 'rating',
       'unit_price', 'unit', 'aisle', 'free_returns', 'sizes', 'colors',
       'seller', 'other_attributes', 'customer_reviews', 'ingredients',
       'initial_price', 'discount', 'ingredients_full', 'categories'],
      dtype='object')

In [24]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 44 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   timestamp               1000 non-null   object 
 1   url                     1000 non-null   object 
 2   final_price             1000 non-null   float64
 3   sku                     1000 non-null   int64  
 4   currency                1000 non-null   object 
 5   gtin                    999 non-null    float64
 6   specifications          1000 non-null   object 
 7   image_urls              1000 non-null   object 
 8   top_reviews             1000 non-null   object 
 9   rating_stars            1000 non-null   object 
 10  related_pages           991 non-null    object 
 11  available_for_delivery  1000 non-null   bool   
 12  available_for_pickup    1000 non-null   bool   
 13  brand                   1000 non-null   object 
 14  breadcrumbs             1000 non-null   o

In [25]:
data.describe()

,final_price,sku,gtin,review_count,product_id,upc,rating,unit_price,initial_price
count,1000.00000,1.000000e+03,9.990000e+02,1000.000000,1.000000e+03,9.990000e+02,1000.000000,101.000000,898.000000
mean,37.98634,2.203601e+09,5.786768e+11,465.027000,2.203601e+09,5.786768e+11,4.338800,12.234287,61.535278
std,97.48094,1.987681e+09,2.640686e+11,2049.842999,1.987681e+09,2.640686e+11,0.703825,30.482208,165.317442
min,0.91000,1.029102e+07,1.111102e+10,1.000000,1.029102e+07,1.111102e+10,1.000000,0.083000,1.580000
25%,13.96000,5.576833e+08,4.620225e+11,9.000000,5.576833e+08,4.620225e+11,4.200000,0.624000,19.990000
50%,19.99000,1.446916e+09,7.024018e+11,47.000000,1.446916e+09,7.024018e+11,4.500000,1.810000,33.590000
75%,35.97000,3.769058e+09,7.651449e+11,206.500000,3.769058e+09,7.651449e+11,4.700000,10.500000,55.000000
max,1685.99000,6.925859e+09,9.931881e+11,43691.000000,6.925859e+09,9.931881e+11,5.000000,199.250000,2629.990000


In [26]:
data.isnull().sum()

timestamp                   0
url                         0
final_price                 0
sku                         0
currency                    0
gtin                        1
specifications              0
image_urls                  0
top_reviews                 0
rating_stars                0
related_pages               9
available_for_delivery      0
available_for_pickup        0
brand                       0
breadcrumbs                 0
category_ids                0
review_count                0
description                 1
product_id                  0
product_name                0
review_tags                 0
category_url                0
category_name               0
category_path               0
root_category_url           0
root_category_name          0
upc                         1
tags                      213
main_image                  0
rating                      0
unit_price                899
unit                      899
aisle                     950
free_retur

In [27]:
data['product_name'].nunique()

968

In [28]:
data['category_name'].value_counts()

category_name
Plus Size Tops            33
Shop Curtains             26
Cooling Sheets            20
Colored Sheets            17
Womens Blouses            17
                          ..
Blackout Curtains          1
Native Deodorant           1
Football Pants             1
Mens Regular Fit Jeans     1
Retractable baby gates     1
Name: count, Length: 452, dtype: int64

### Identify essential columns

In [29]:
if 'product_name' in data.columns:
    name_column = 'product_name'
elif 'name' in data.columns:
    name_column = 'name'
else:
    name_column = data.select_dtypes(include=['object']).columns[0]

data[name_column].dtype


dtype('O')

In [30]:
if 'product_description' in data.columns:
    description_column = 'product_description'
elif 'description' in data.columns:
    description_column = 'description'
else:
    description_column = data.select_dtypes(include=['object']).columns[1]

### Cleaning Text

In [31]:
# A function to clean the text data and use regex to remove unwanted characters
def clean_text(text):
    if pd.isnull(text):
        return ""
    text = text.lower()
    text = re.sub(r'[^\w\s]', '', text)
    return text.strip()

In [32]:
data['cleaned_desc'] = data[description_column].apply(clean_text)

### Using TFIDFVectorizer

In [33]:
tfidf = TfidfVectorizer(stop_words='english', max_features=10000)
tfidf_matrix = tfidf.fit_transform(data['cleaned_desc'])
tfidf_matrix.shape

(1000, 10000)

In [34]:
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(1000, 1000)

In [35]:
indices = pd.Series(data.index, index=data[name_column].astype(str).str.lower()).drop_duplicates()

In [37]:
#Recommendation function
def recommend_product(product_name, no_of_recommendations=5):
    key = product_name.lower()
    if key not in indices:
        return f"'{product_name}' not found in product list."
    idx = indices[key]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:no_of_recommendations+1]
    product_indices = [i[0] for i in sim_scores]
    return data.iloc[product_indices][[name_column, description_column]]


In [38]:
#Usage :
sample_product = data[name_column].iloc[0]
print(f"Sample Product: {sample_product}")
recommendations = recommend_product(sample_product, 5)
print(f"Recommendations: {recommendations}")

Sample Product: Laura Mercier Caviar Stick Eye Color Sugar Frost 1.64g/0.05oz
Recommendations:                                           product_name  \
928       NYX Jumbo Eye Pencil (Color : #610 - Oyster)   
396  Pretty Smart Celebrity Artist Eyeshadow Palett...   
215  Badger Kids Sunscreen Stick SPF 35 with Minera...   
26   Anastasia Beverly Hills Norvina PRO Pigment Ey...   
5    Women's Vanity Fair 75201 Nearly Invisible Ful...   

                                           description  
928  What it is: A radiant eye liner that also doub...  
396  Pretty Smart offers high-quality, luxury inspi...  
215     SAFE EFFECTIVE PROTECTION: Protect your chi...  
26   Anastasia Beverly Hills ABH Norvina Pro Pigmen...  
5       Amazing comfort in this seamless T-shirt br...  


### Saving the model

In [39]:
import joblib
joblib.dump(tfidf, 'tfidf_model.pkl')
joblib.dump(tfidf_matrix, 'tfidf_matrix.pkl')
data.to_csv('walmart_products_cleaned.csv', index=False)
print("Model and data saved successfully.")

Model and data saved successfully.
